In [ ]:
# Tinnitus project phase reset/erp
!pip install EMD-signal

from google.colab import drive
drive.mount('/content/gdrive')

import numpy as np
from numpy import exp
import dill    # pkl eval
from scipy.io import loadmat
import matplotlib.pyplot as plt
from scipy.signal import hilbert
import scipy.ndimage
import scipy as sp
from scipy import signal
import time

import pandas as pd
from PyEMD import CEEMDAN, EMD, Visualisation

## Tina put the parameters that you like here ##
# Stimulus Frequency # i have to figure out how to find an elegant way to include all of them
# stim_freq = 3000 # be careful leave the ' it is a string not an integer '3000' '5000'...
stim_freqs = np.array([7000, 9000, 11000])
print('Stimulus frequency ', stim_freqs)
# Cz index from 1 to 69 (or whatever)
cz_index = 69

# just testing the other file, this one might be corrupted
data = loadmat('gdrive/My Drive/Experiment 12-08-19/12-8-19 experiment analyzed on 1-17-20.mat')
channels = data['EEG']['data'][0][0]
data_l = loadmat('gdrive/My Drive/Experiment 12-08-19/labels.mat')
labels = data_l['labels'][0]

data_times = loadmat('gdrive/My Drive/Experiment 12-08-19/tinatimes.mat')
stims = data_times['C'][0]

# Give the location of the file
print('Data size: ', channels.shape[0], channels.shape[1])
# cz is 69th? like always?
cz = np.squeeze(channels[cz_index-1, :])
print(cz)

# window
x = np.arange(-100,2000)

# take first 1.1s (100s before 1000ms after)
# plt.imshow(cz[0:1100].transpose(), aspect='auto', cmap='jet') #, vmin=0, vmax=np.max(y))  # , aspect='auto'
# plt.colorbar()
# plt.show()  

fs = 1000
order = 2
if (1):

  for stim_freq in stim_freqs:
    n = np.sum(1*(labels == stim_freq))
    cz_stim = np.zeros((2100, 7, n))

    loc_cnti = 0
    for cnti, val in enumerate(stims[0:-2]):

      ti = int(stims[cnti]*1000) #-1
      if labels[cnti] == stim_freq:
        
        S = cz[ti-2000:ti+4000]
        print('emd')
        components = CEEMDAN()(S)
        components = components[0:7]

        if 0:
          imfs, res = components[:-1], components[-1]
          t = np.arange(0, components[0], 1)
          vis = Visualisation()
          vis.plot_imfs(imfs=imfs, residue=res, t=t, include_residue=True)
          vis.plot_instant_freq(t, imfs=imfs)
          vis.show()
        
        for cntc, val in enumerate(components):
          #start = time.time()
          mode = components[cntc, 1900:4000]
          y = hilbert(mode) #imfs
          angles = np.angle(y)
          insta_phase = np.unwrap(angles)  # should we ingore this and go straight to the normsss
          insta_phase_norms = (insta_phase + np.pi) / (2 * np.pi) % 1.
          #end = time.time()
          #print(end - start)
          cz_stim[:, cntc, loc_cnti] = insta_phase_norms
        loc_cnti += 1


    # window
    x = np.arange(-100,2000)

    for cnts in range(0, cz_stim.shape[1]):

      cz_sti = np.squeeze(cz_stim[:, cnts, :])
      nbins = 200
      hist_phases = np.zeros([nbins, cz_sti.shape[0]])
      for cntt in range(cz_stim.shape[0]):
        test = np.histogram(cz_sti[cntt, :], nbins, (0, 1))  # calc hist wind_[ind, :]
        hist_phases[:, cntt] = test[0]

      sigma_y = 2.0
      sigma_x = 2.0
      # Apply gaussian filter
      sigma = [sigma_y, sigma_x]
      y = sp.ndimage.filters.gaussian_filter(hist_phases, sigma, mode='constant')

      fig, ax = plt.subplots(1,1)
      img = ax.imshow(y[:, 0:1100])

      x_label_list = ['-100', '0', '500', '1000']
      ax.set_xticks([0,100,600,1100])
      ax.set_xticklabels(x_label_list)

      y_label_list = ['0', '0.5', '1']
      ax.set_yticks([0,100,199])
      ax.set_yticklabels(y_label_list)

      divider = make_axes_locatable(ax)
      cax = divider.append_axes("right", size="5%", pad=0.05)
      fig.colorbar(img, cax=cax)
      plt.show()
      plt.savefig('histogram_phases', bbox_inches = 'tight', pad_inches = 0)

    #params = paramsClass()